In [2]:
import pandas as pd
import seaborn as sns
import igraph

%matplotlib inline

ModuleNotFoundError: No module named 'igraph'

In [ ]:
g = igraph.load('florida_coa_graph’,format='pickle’)

#Here is how to map for to get authors / papers

method = ‘florida'
a_2_node = np.load('/%s/data/%s_a_2_node.npy'%(homedir,method),allow_pickle='TRUE').item()
a_2_paper = np.load('/%s/data/%s_a_2_paper.npy'%(homedir,method),allow_pickle='TRUE').item()
node_2_a = np.load('/%s/data/%s_node_2_a.npy'%(homedir,method),allow_pickle='TRUE').item()